# VTune™ Profiling on Intel® DevCloud
#### Based on Intel training module, revised for EECE.4510/5510 Labs
##### by Prof. Yan Luo. June 12, 2023.

##### Sections
- [What is VTune™ Profiler?](#What-is-VTune™-Profiler?)
- [VTune™ Command-line Options](#VTune™-Command-line-Options)
- _Lab Exercise:_ [VTune™ gpu_hotspots profiling with iso3dfd sample](#Lab-Exercise:-VTune™-Profiling)

## Learning Objectives

- Profile a SYCL application using the VTune™ profiling tool on Intel® DevCloud
- Understand the basics of VTune™ command line options for collecting data and generating reports

## What is VTune™ Profiler?

VTune™ allows SYCL Profiling capabilities so you can tune for CPU, GPU, and FPGA.

![VTune UI](vtuneui.png)

__Analyze Data Parallell C++__ :
See the lines of DPC++ that consume the most time

__Tune for CPU, GPU & FPGA__ :
Optimize for any supported hardware accelerator

__Optimize Offload__ :
Tune OpenMP offload performance

__Wide Range of Performance Profiles__ :
CPU, GPU, FPGA, threading, memory, cache, storage…

__Most Popular Languages__ :
SYCL, C, C++, Fortran*, Python*, Go*, Java*, or a mix


## VTune™ Command-line Options

### Run and collect VTune™ data
```vtune -collect gpu_hotspots -result-dir vtune_data a.out```

Various types of profiling data can be collected like `hotspots`, `memory-consumption`, `memory-access`, `threading`…

Use the command line help to find out more:

```vtune --help -collect```

### Generate html report for collected VTune™ data:
```vtune -report summary -result-dir vtune_data -format html -report-output $(pwd)/summary.html```

Various types of report can be generated like `summary`, `top-down`, `callstacks`…

Use the command line help to find out more:

```vtune --help -report```


## When to use VTune™ Command line

VTune™ Command-line is useful when on __Intel® DevCloud__ or you only have __SSH__ access to development system.

However, it is recommended to install the __full VTune™ version__ on a local system and use the __UI rich experience__ of VTune Profiling Tool.

![VTune UI](vtuneui.png)


## Lab Exercise: VTune™ Profiling
- Build, run, collect VTune™ data and display VTune summary when running on gpu and cpu.

### Test Application: SYCL implementation of iso3dfd

SYCL implementation of iso3dfd will be used to collect VTune™ data and analyze the generated result. Below are source code to iso3dfd application:
- [iso3dfd.cpp](src/iso3dfd.cpp)
- [iso3dfd_kernels.cpp](src/iso3dfd_kernels.cpp)


### Build and Run

In [1]:
%%writefile run_iso3dfd.sh
#!/bin/bash
source /opt/intel/inteloneapi/setvars.sh > /dev/null 2>&1

icpx -fsycl src/iso3dfd.cpp src/utils.cpp src/iso3dfd_kernels.cpp -o iso3dfd

#TODO: EECE.5510
#TODO: uncomment the following to use shared local memory in the kernel function
#TODO: build and then run vtune again to observe the difference in performance profiling results
icpx -fsycl -DUSE_SHARED src/iso3dfd.cpp src/utils.cpp src/iso3dfd_kernels.cpp -o iso3dfd

./iso3dfd 256 256 256 8 8 8 20 sycl gpu



Overwriting run_iso3dfd.sh


**STEP 1:** Build and Run the iso3dfd app by running ▶ the command below:

In [2]:
! chmod 755 q; chmod 755 run_iso3dfd.sh; if [ -x "$(command -v qsub)" ]; then ./q run_iso3dfd.sh; else ./run_iso3dfd.sh; fi

Job has been submitted to Intel(R) DevCloud and will execute soon.

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2324811.v-qsvr-1           ...ub-singleuser u193166         00:01:27 R jupyterhub     
2324916.v-qsvr-1           run_iso3dfd.sh   u193166                0 Q batch          

Waiting for Output ██████████████████████████████████████████████ Done⬇

########################################################################
#      Date:           Wed 21 Jun 2023 02:19:17 PM PDT
#    Job ID:           2324916.v-qsvr-1.aidevcloud
#      User:           u193166
# Resources:           cput=75:00:00,neednodes=1:gpu:ppn=2,nodes=1:gpu:ppn=2,walltime=06:00:00
########################################################################

Grid Sizes: 256 256 256
Memory Usage: 230 MB
 ***** Running SYCL variant *****
Initializing ... 
 Running on Intel(R) UHD Graphics [0x9a60]
 The Device M

## Analyze performance with VTune™

Use VTune™ command line to analyze performace on GPU vs CPU and display the summary

### VTune™ Command Line for collecting and reporting

In [3]:
%%writefile vtune_collect.sh
#!/bin/bash
source /opt/intel/inteloneapi/setvars.sh
/bin/echo "##" $(whoami) is compiling SYCL_Essentials Module6 -- Intel Vtune profiler - 1 of 1 Vtune_Profiler
#vtune
#type=hotspots
#type=memory-consumption
#type=uarch-exploration
#type=memory-access
#type=threading
#type=hpc-performance
#type=system-overview
#type=graphics-rendering
#type=io
#type=fpga-interaction
#type=gpu-offload
type=gpu-hotspots
#type=throttling
#type=platform-profiler
#type=cpugpu-concurrency
#type=tsx-exploration
#type=tsx-hotspots
#type=sgx-hotspots

rm -r vtune_data

# added by yluo
echo "stop prior collection if any"
vtune -command stop

echo "Vtune Collect $type"
vtune -collect $type -result-dir vtune_data $(pwd)/iso3dfd 256 256 256 8 8 8 20 sycl gpu

echo "Vtune Summary Report"
#vtune -report summary -result-dir vtune_data -format html -report-output $(pwd)/summary.html
vtune -report summary -result-dir vtune_data -format html -report-output ./summary.html


Overwriting vtune_collect.sh


### Run VTune™ to Collect Hotspots and Generate Report

**STEP 2:** Collect VTune™ data and generate report by running ▶ the command below:

In [ ]:
! chmod 755 vtune_collect.sh; if [ -x "$(command -v qsub)" ]; then ./q vtune_collect.sh; else ./vtune_collect.sh; fi

Job has been submitted to Intel(R) DevCloud and will execute soon.

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2324811.v-qsvr-1           ...ub-singleuser u193166         00:01:31 R jupyterhub     
2324917.v-qsvr-1           vtune_collect.sh u193166                0 Q batch          

Waiting for Output ███████████████████████████████████████████████████

## Display VTune™ Summary

Display VTune™ summary report generated in html format

### Display VTune™ Report for GPU

**STEP 3:** Display VTune™ summary report by running ▶ the command below 

In [ ]:
from IPython.display import IFrame
IFrame(src='summary.html', width=960, height=600)

### <font color='red'>Question</font>
Change the script in Build and Run to add "-DUSE_SHARED" flag such that the kernel function will use local shared memory. Run through the build, run, profile process again. Observe the difference in profiling results. Capture and explain your findings in your report.

## Summary

VTune™ command line is useful for quick analysis of SYCL application to get performance metric and tune applications.